In [1]:
from pyscipopt import Model, quicksum, multidict
from cflp_function import *
from calculate_od import *

In [2]:
folder_path = "app_data"
I = load_data_from_pickle(folder_path, 'Farm_test.pickle')
d = load_data_from_pickle(folder_path, 'manure_production_test.pickle')

In [3]:
sum(d.values())

704905

In [4]:
def load_csv(csv_path):
    df = pd.read_csv(csv_path)
    return df
def load_gdf(gdf_path):
    gdf = gpd.read_file(gdf_path)
    return gdf

In [24]:
main_crs = "EPSG:4326"

In [68]:
### LOAD DATA ###
loi = load_csv('./hex/loi.csv') # Mock candidate sites
farm_gdf = load_gdf("./farm/farm_new.shp")
n = load_gdf("./osm_network/G_n.shp") # Road network nodes
n = n.to_crs(main_crs)
### CALCULATE OD MATRIX ###
loi.index = range(1, len(loi) +1) # Reset index to start with 1 (because users don't like 0 as the 1st value...)
loi_gdf = loi_to_gdf(loi) # Find centroid of hexagons and convert to gdf
loi_gdf['y'] = loi_gdf['geometry'].y
loi_gdf['x'] = loi_gdf['geometry'].x
find_closest_osmid(farm_gdf, n)
find_closest_osmid(loi_gdf, n)
c, plant = calculate_od_matrix(farm_gdf, loi_gdf, cost_per_km=0.69, frequency_per_day=1, lifetime_in_days=1)

In [69]:
M, f = assign_capacity_capex(plant)

In [70]:
J = plant

In [105]:
p=0.2

In [143]:
model = Model("flp_percentage_demand")

x, y, z = {}, {}, {}
total_demand = sum(d[i] for i in I)
target_demand = total_demand * p

for j in J:
    y[j] = model.addVar(vtype="B", name="y(%s)" % j)
    for i in I:
        x[i, j] = model.addVar(vtype="C", name="x(%s,%s)" % (i, j))
        z[i, j] = model.addVar(vtype="B", name="z(%s,%s)" % (i, j))
# # Enforce z[i, j] = 1 if x[i, j] > 0
# for (i, j) in x:
#     model.addCons(z[i, j] >= x[i, j] / d[i], "EnforceZCondition(%s,%s)" % (i, j))

for i in I:
    model.addCons(quicksum(x[i, j] for j in J) == d[i] * z[i, j], "Demand(%s)" % i)

for j in M:
    model.addCons(quicksum(x[i, j] for i in I) <= M[j] * y[j], "Capacity(%s)" % j)

for (i, j) in x:
    model.addCons(x[i, j] <= d[i] * y[j], "Strong(%s,%s)" % (i, j))

model.addCons(quicksum(x[i, j] for i in I for j in J) >= target_demand, "PercentageDemand")

model.setObjective(
    quicksum(f[j] * y[j] for j in J) +
    quicksum(c[i, j] * z[i, j] for i in I for j in J),
    "minimize"
)
model.data = x, y, z

In [141]:
model.optimize()

In [142]:
model.getObjVal()

37311300.674544945

In [129]:
# # Print the values of y, x, and z in the solution
# for j in J:
#     print(f"y[{j}] =", model.getVal(y[j]))

# for i in I:
#     for j in J:
#         print(f"x[{i},{j}] =", model.getVal(x[i, j]))
#         print(f"z[{i},{j}] =", model.getVal(z[i, j]))

In [130]:
EPS = 1.e-6
x,y,z = model.data
assignment = [(i,j) for (i,j) in x if model.getVal(x[i,j]) > EPS]
facilities = [j for j in y if model.getVal(y[j]) > EPS]

In [131]:
facilities

[1, 7]

In [132]:
assignment

[(53, 1),
 (54, 1),
 (65, 1),
 (66, 1),
 (76, 1),
 (98, 1),
 (116, 1),
 (118, 1),
 (136, 1),
 (137, 1),
 (138, 1),
 (140, 1),
 (141, 1),
 (5, 7),
 (6, 7),
 (24, 7),
 (100, 7),
 (119, 7),
 (133, 7),
 (135, 7)]

In [148]:
# Create a dictionary to store the results
result_dict = {f: [] for f in facilities}
# Iterate over edges and populate the result_dict
for (i, j) in assignment:
    if j in facilities:
        result_dict[j].append(i)

In [149]:
result_dict

{1: [53, 54, 65, 66, 76, 98, 116, 118, 136, 137, 138, 140, 141],
 7: [5, 6, 24, 100, 119, 133, 135]}

In [144]:
total_c = sum(c[key] for key in assignment if key in c)*365*12

In [145]:
total_c

1573647.31506509

In [146]:
# Example dictionary
result_dict = {(1, 2): 10, (3, 4): 20, (5, 6): 30}

# Example values
total_capex = len(result_dict) * 6089160
total_opex = len(result_dict) * (1047200 * 12)

# Sum the values corresponding to the keys in result_dict
total_c = sum(result_dict[key] for key in result_dict)

# Create a DataFrame
data = {'Category': ['Total C', 'Total Capex', 'Total Opex'],
        'Value': [total_c, total_capex, total_opex]}

df = pd.DataFrame(data)

# Display the DataFrame
print(df)

      Category     Value
0      Total C        60
1  Total Capex  18267480
2   Total Opex  37699200


In [147]:
sum(df['Value'])

55966740

In [135]:
# Example list of keys
keys_to_search = [(1, 2), (3, 4)]

# Example dictionary
another_dict = {(1, 2): 10, (3, 4): 20, (5, 6): 30}

# Sum the values corresponding to the keys in the list
total_sum = sum(another_dict[key] for key in keys_to_search if key in another_dict)

print("Total Sum:", total_sum)

Total Sum: 30


In [114]:
[(i,j) for (i,j) in z if model.getVal(z[i,j]) > EPS]

[(5, 7),
 (6, 7),
 (24, 7),
 (53, 7),
 (54, 7),
 (65, 7),
 (66, 7),
 (76, 7),
 (98, 7),
 (100, 7),
 (116, 7),
 (118, 7),
 (119, 7),
 (133, 7),
 (135, 7),
 (136, 7),
 (137, 7),
 (138, 7),
 (140, 7),
 (141, 7)]

In [138]:
total_cost = model.getObjVal()

# Create a dictionary to store the results
result_dict = {f: [] for f in facilities}
# Iterate over edges and populate the result_dict
for (i, j) in assignment:
    if j in facilities:
        result_dict[j].append(i)

# Get percentage of utilization
x_values = {(i, j): model.getVal(x[i, j]) for (i, j) in x if model.getVal(x[i, j]) > EPS} # get how much is flowing between every assignment
flow_matrix = np.array([[x_values.get((i, j), 0) for j in J] for i in I]) # create a flow matrix (len(farm)xlen(plant))
column_sum = np.sum(flow_matrix, axis=0) # sum of total flow going to every plant
used_capacity = (column_sum/np.array(list(M.values())))*100

In [79]:
used_capacity_df = pd.DataFrame(used_capacity, index=J)

In [80]:
used_capacity_df

,0
1,0.000000
2,0.000000
3,0.000000
4,58.991024
5,0.000000
6,0.000000
7,0.000000


In [81]:
# Example DataFrames
farm_df = pd.DataFrame({
    'color': ['red', 'blue', 'green', 'yellow'],
    # ... other columns in farm_df
})

digester_df = pd.DataFrame({
    'color': [
        [255, 165, 0],  # orange
        [128, 0, 128],  # purple
        [165, 42, 42],  # brown
        [255, 192, 203]  # pink
    ],
    # ... other columns in digester_df
})

assignment_decision = {
    0: [1, 2],
    1: [0, 3],
    # ... other keys and values in assignment_decision
}


In [82]:
# Update farm_df based on assignment_decision
for digester_index, farm_indices in assignment_decision.items():
    digester_color = digester_df.loc[digester_index, 'color']
    print(farm_indices)
    farm_df.loc[farm_indices, 'color'] = str(digester_color)

# Now, farm_df has been updated based on digester_df and assignment_decision
print(farm_df)

[1, 2]
[0, 3]
           color
0  [128, 0, 128]
1  [255, 165, 0]
2  [255, 165, 0]
3  [128, 0, 128]


In [83]:
digester_df

,color
0,"[255, 165, 0]"
1,"[128, 0, 128]"
2,"[165, 42, 42]"
3,"[255, 192, 203]"


In [84]:
farm_df

,color
0,"[128, 0, 128]"
1,"[255, 165, 0]"
2,"[255, 165, 0]"
3,"[128, 0, 128]"
